<a href="https://colab.research.google.com/github/RomalPatel28/Romal_INFO5731_Fall2021/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(20 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect all User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the abstracts research papers by using the query [smart health](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 10,000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter with the following requirements: Location (Texas), Time frame (2021-01-01 to 2021-09-01). 


In [12]:
# Write your code here
# importing neccessary packages
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import urllib.request

url = "https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc"
page = urllib.request.urlopen(url)

#Parse the html in the 'page' variable and store it in Beautiful Soup
soup = BeautifulSoup(page)

title = []
for new_title in soup.find_all('a',{'class':'remove doc_details'}):
  new_title = title.append(new_title.text.strip())

year = []
for new_year in soup.find_all("span",{'class':'pubyear'}):
  new_year = year.append(new_year.text.strip())

authors = []
for author_name in soup.find_all('span',{'class':'authors'}):
  author_name = authors.append(author_name.text.strip())

abstract = []
for sub in soup.find_all("div",{'class': 'snippet'}):
  sub = abstract.append(sub.text.strip())

new_dict = {'Title':title,'Year':year,'Authors':authors,'Abstract':abstract}

NLParticles = pd.DataFrame.from_dict(new_dict, orient='index')
NLParticles.transpose()

new_file = NLParticles.to_csv("NLP_articles.csv")
NLParticles




,0,1,2,3,4,5,6,7,8,9
Title,Foundations of statistical natural language pr...,A Maximum Entropy approach to Natural Language...,Natural Language Processing,Linguistics and Natural Language Processing,Natural Language Processing,Natural language processing (almost) from scratch,Natural language processing,Natural Language Processing/Robotics,Tutorial on Natural Language Processing,Ambiguities in Natural Language Processing
Year,", 2000",", 1996",", 2007",", 1987",", 2001",", 2011",", 1996",None,None,None
Authors,"by \n Christopher D. Manning,...","by \n Adam L. Berger, Stephen...","by \n Cohn Trevor A, Trevor A...",by \n Victor Raskin,by \n Enrico Franconi,"by \n Ronan Collobert, Jason ...","by \n Mark Steedman, Linc Lab...",by \n Timothy Brick,by \n unknown authors,"by \n Anjali M , Babu Anto"
Abstract,"""... ...""","""... The concept of maximum entropy can be tra...","""... Scaling conditional random fields for nat...","""... The paper addresses the issue of cooperat...","""... In most natural language processing appli...","""... We propose a unified neural network archi...","""... Natural Language Processing The subject o...","""... Robots that interact with humans face-to-...","""... Natural languages are languages spoken by...","""... ABSTRACT: Ambiguity can be referred as t..."


# **Question 2**

(30 points) Here is a [legal case](https://github.com/unt-iialab/info5731-fall2021/blob/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



(1) Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

(2) Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

(3) Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


(4) Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


In [31]:
# Write your code here
# 2.
import pandas as pd
dataframe = pd.read_csv("legal_case.txt", sep="\n")
dataframe.rename(columns = {'5 Ala. 740':'legal_cases'}, inplace = True)

# Number of words
dataframe['word_count'] = dataframe['legal_cases'].apply(lambda x: len(str(x).split(" ")))

# character count
dataframe['char_count'] = dataframe['legal_cases'].str.len()

# Averge word length
def average(sentence):
  words = sentence.split()
  try:
    return (sum(len(i) for i in words)/len(words))
  except ZeroDivisionError:
    words = 0 

dataframe['average word'] = dataframe['legal_cases'].apply(lambda x: average(x))

# Uppercase
dataframe['upper'] = dataframe['legal_cases'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
dataframe


,legal_cases,word_count,char_count,average word,upper
0,Supreme Court of Alabama.,4,25,5.500000,0
1,ADAMS,1,5,5.000000,1
2,v.,1,2,2.000000,0
3,TANNER AND HORTON.,3,18,5.333333,3
4,"June Term, 1843.",3,16,4.666667,0
...,...,...,...,...,...
142,There are no Filings for this citation.,7,39,4.714286,0
143,Negative Treatment,2,18,8.500000,0
144,There are no Negative Treatment results for th...,9,58,5.555556,0
145,History,1,7,7.000000,0


,legal_cases,word_count,char_count
0,Supreme Court of Alabama.,4,25
1,ADAMS,1,5
2,v.,1,2
3,TANNER AND HORTON.,3,18
4,"June Term, 1843.",3,16
...,...,...,...
142,There are no Filings for this citation.,7,39
143,Negative Treatment,2,18
144,There are no Negative Treatment results for th...,9,58
145,History,1,7


In [26]:
# Lower case
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# Removing Punctuations
import string
dataframe['legal_cases'] = dataframe['legal_cases'].str.replace('[{}]'.format(string.punctuation), '')

# Removing Stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop=stopwords.words('english')
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

# frequent word removal
freq=pd.Series(' '.join(dataframe['legal_cases']).split()).value_counts()[:10]
freq=list(freq.index)
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x:" ".join(x for x in x.split() if x not in freq))

# Rar word removal
freq=pd.Series(' '.join(dataframe['legal_cases']).split()).value_counts()[-10:]
freq=list(freq.index)
dataframe['Legal_cases'] = dataframe['legal_cases'].apply(lambda x:" ".join(x for x in x.split() if x not in freq))

# Spelling Correction
from textblob import TextBlob
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x: str(TextBlob(x).correct()))

# Stemming
from nltk.stem import PorterStemmer
st=PorterStemmer()
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Lemmatization
from textblob import Word
nltk.download('wordnet')
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# Tokenization
import nltk
nltk.download('punkt')
dataframe['legal_cases'] = dataframe['legal_cases'].apply(lambda x: TextBlob(x).words)

# (3) Saving to a csv file
dataframe.to_csv('processed_file.csv')
print(dataframe)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
                          legal_cases  ...                          Legal_cases
0                   [suprem, alabama]  ...                      supreme alabama
1                              [adam]  ...                                adams
2                                  []  ...                                     
3                    [manner, norton]  ...                        tanner horton
4                        [june, 1843]  ...                            june 1843
..                                ...  ...                                  ...
142                    [file, situat]  ...                     filings citation
143                  [neg, t

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here






**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
# You explaination here





# **Question 4**

(20 points) Python Regular Expression.

(1) Write a Python program to remove leading zeros from an IP address.

ip = "260.08.094.109"


(2) Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [6]:
# 1. Using a function to keep track of the zeros
ipadd = "260.08.094.109" 
def original_sentence(ipadd):
  no_leading0 = ".".join([str(int(j)) for j in ipadd.split(".")]) 
  return no_leading0;

print("The new IP address after removing the leading zeroes is: " + original_sentence(ipadd))

# 2. Extract all the years from the following sentence
import re
year_sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
no_year = re.findall('2\d{3}', year_sentence)
print("The extracted years are: " + str(no_year))






The new IP address after removing the leading zeroes is: 260.8.94.109
The extracted years are: ['2010', '2010', '2019']
